### Solution strategies used for an example problem of finding a sequence of distinct primes such that the arithmetic mean of any two of these primes is also a prime as a PP.

In [1]:
import numpy as np
import sympy
import gurobipy as gp
from gurobipy import GRB
import time
import pandas as pd

In [2]:
def PrimeBranching(x, name = 'Naive', arg = None):
    '''
    *** Make sure that x.lb >= arg[1] (k the remainder) ***
    Find an element of list x which has the highest distance from its closest prime number
    If name = Naive; do choose non-prime which is furthest, name = Modulo, arg = k branch to prime of the form 12*m+k
    
    Input argument: solution from the model m; m.x
    arg = [n,k] n: number of solution, k residue modulo 12
    
    Output: index of the branching variable
    '''
    dist = [-1 for _ in x]
    if name == 'Naive':
        for ind, val in enumerate(x):
            round_val = int(round(val,0))
            if sympy.isprime(round_val):
                dist[ind] = abs(val - round_val)
            else:
                next_prime = sympy.nextprime(val)
                prev_prime = sympy.prevprime(val)
                dist[ind] = min(val - prev_prime, next_prime - val)
    elif name == 'Modulo':
        n,k = arg
        k0 = k%6
        for ind, val in enumerate(x):
            # x variable
            if ind < n:
                round_val = int(round(val,0))
                if sympy.isprime(round_val) and (round_val%12 == k):
                    dist[ind] = abs(val - round_val)
                else:
                    next_prime = sympy.nextprime(val)
                    while(next_prime%12 != k):
                        next_prime = sympy.nextprime(next_prime + 1)
                    prev_prime = sympy.prevprime(val)
                    while(prev_prime%12 != k):
                        prev_prime = sympy.prevprime(prev_prime - 1)
                    dist[ind] = min(val - prev_prime, next_prime - val)
            else:
                round_val = int(round(val,0))
                if sympy.isprime(round_val) and (round_val%6 == k0):
                    dist[ind] = abs(val - round_val)
                else:
                    next_prime = sympy.nextprime(val)
                    while(next_prime%6 != k0):
                        next_prime = sympy.nextprime(next_prime + 1)
                    prev_prime = sympy.prevprime(val)
                    while(prev_prime%6 != k0):
                        prev_prime = sympy.prevprime(prev_prime - 1)
                    dist[ind] = min(val - prev_prime, next_prime - val)
    br_ind = np.argmax(dist)
    return br_ind

In [3]:
def Branch_val(x, br_ind, name = 'Naive', arg = None):
    '''
    Output: branch_down, branch_up := branching values
    '''
    if name == 'Naive':
        branch_down = sympy.prevprime(x[br_ind])
        branch_up = sympy.nextprime(x[br_ind])
    elif name == 'Modulo':
        n,k = arg
        k0 = k%6
        if br_ind < n:
            prev_prime = sympy.prevprime(x[br_ind])
            while(prev_prime%12 != k):
                prev_prime = sympy.prevprime(prev_prime - 1)
            branch_down = prev_prime
            next_prime = sympy.nextprime(x[br_ind])
            while(next_prime%12 != k):
                next_prime = sympy.nextprime(next_prime + 1)
            branch_up = next_prime
        else:
            prev_prime = sympy.prevprime(x[br_ind])
            while(prev_prime%6 != k0):
                prev_prime = sympy.prevprime(prev_prime - 1)
            branch_down = prev_prime
            next_prime = sympy.nextprime(x[br_ind])
            while(next_prime%6 != k0):
                next_prime = sympy.nextprime(next_prime + 1)
            branch_up = next_prime
    return branch_down, branch_up

In [4]:
def testInteger(x):
    '''
    Test if the given list of number are all integers
    
    Input argument: x (list of numbers)
    
    Output: Boolean, if all elements of x are integer or not
    '''
    test = np.round(x, 6) - np.round(x, 0)
    test = np.abs(test)
    tolerance = np.ones(len(test)) * -1e-6
    if all(test < tolerance) or all(test == 0):
        return True
    else:
        return False

In [5]:
def testPrime(x, name = 'Naive', arg = None):
    '''
    Naive: test if the given list of number are all prime numbers 
    Modulo: test if the given list of number are all prime numbers of the form 12m+k for the first n terms and 6m+k0 else
    
    Input argument: x (list of numbers)
    
    Output: Boolean, if all elements of x are prime or not
    '''
    
    # we have to first test if it is an integer solution
    if testInteger(x):
        test = np.round(x, 6)
        for val in test:
            if not sympy.isprime(int(val)):
                return False
        return True
    else:
        return False

In [6]:
def check_prime_sol(x):
    # Check if the obtained solution satisfy the conditions
    n = len(x)
    for i in range(n):
        if sympy.isprime(x[i]) == False:
            print(x[i])
            return 0
        for j in range(i+1,n):
            if sympy.isprime(int((x[i]+x[j])/2)) == False:
                print(int((x[i]+x[j])/2))
                return 0
    return 1 

In [7]:
def fixing_strategy(name = 'Naive', x_bar = [], p =[]):
    # p position to be excluded
    # if p > m; do not remove
    # kwargs contain 1. solution from previous iteration 2.position p = [p_1,p_2,...] 3. position l to exclude l = [l_1, l_2,...]
    
    if x_bar == [] or name == 'Naive':
        return []
    if name == 'SelectAll':
        return x_bar
    if name == 'ExcludeOne' or name == 'ExcludeTwo':
        x_prime = []
        for i,x in enumerate(x_bar):
             if i not in p:
                    x_prime.append(x)
        return x_prime

In [8]:
def find_average_prime(n, init_bound = 10**3, upper_bound = 10**3, time_limit = 600, alpha = 0.2, branch_name = 'Naive', remainder = 0, fix_name = 'Naive', prev_sol = [], exclude_pos =[]):
    '''
    Find n primes such that am average of every two primes is also prime
    
    Output: 
        node_count (int): number of nodes in the enumeration tree
        global_ub (float): the optimal objective value
        incumbent (list): prime number solution to the problem
        computation_time (float): computation time of the function in seconds
    
    '''
    start_time = time.time()
    
    # Create optimization problem
    #Suppressing Gurobi output
    env = gp.Env(empty=True)
    env.setParam("OutputFlag",0)
    env.start()
    m = gp.Model("Prime Optimization", env = env)
    
    #Set lower bound for variables
    if branch_name == 'Naive':
        #Variables
        x = m.addVars(n, lb = 3, ub = upper_bound, name = 'x')
        y = m.addVars(n, n, lb = 5, ub = upper_bound, name = 'y')
        for i in range(n):
            for j in range(0,i+1):
                y[i,j].lb = 5
                y[i,j].ub = 5
    elif branch_name == 'Modulo':
        if remainder == 1:
            lower_bound = 13
        elif remainder == 5:
            lower_bound = 5
        elif remainder == 7:
            lower_bound = 7
        elif remainder == 11:
            lower_bound = 11
        x = m.addVars(n, lb = lower_bound, ub = upper_bound, name = 'x')
        y = m.addVars(n, n, lb = lower_bound, ub = upper_bound, name = 'y')
        
        # Set lower bound to be smallest prime of the form 12m + remainder
        for i in range(n):
            for j in range(0,i+1):
                if remainder == 1:
                    y[i,j].lb = 13
                    y[i,j].ub = 13
                elif remainder == 5:
                    y[i,j].lb = 5
                    y[i,j].ub = 5
                elif remainder == 7:
                    y[i,j].lb = 7
                    y[i,j].ub = 7
                elif remainder == 11:
                    y[i,j].lb = 11
                    y[i,j].ub = 11
                else:
                    # print("Problem is infeasible!")
                    return (0,0,[],0)
    
    # Fix varaibles w.r.t. fixing strategy
    # Extract x_bar, p from kwargs
    fix_val = fixing_strategy(name = fix_name, x_bar=prev_sol, p=exclude_pos)
    for i in range(len(fix_val)):
        x[i].lb = fix_val[i]
        x[i].ub = fix_val[i]
        
    # Increase the bound if the last element is free
    if upper_bound > init_bound:
        x[n-1].lb = upper_bound/(1+alpha)
    
    # add constraints
    m.addConstrs(2*y[i,j] == x[i] + x[j] for i in range(n) for j in range(i+1, n))
    if branch_name == 'Modulo':
        m.addConstrs(x[i+1] >= x[i] + 12 for i in range(n-1))
        m.addConstrs(y[i,j] >= x[i] + 6 for i in range(n) for j in range(i+1,n))
        m.addConstrs(y[i,j] <= x[j] - 6 for i in range(n) for j in range(i+1,n))
    else:
        m.addConstrs(x[i+1] >= x[i] + 4 for i in range(n-1))
        m.addConstrs(y[i,j] >= x[i] + 2 for i in range(n) for j in range(i+1,n))
        m.addConstrs(y[i,j] <= x[j] - 2 for i in range(n) for j in range(i+1,n))
    
    # set objective
    m.setObjective(0, GRB.MINIMIZE)

    # optimize
    m.optimize()
    
    #Check if feasible
    if m.status !=2:
        # print("Problem is infeasible!")
        return (0,0,[],0)
    else:
        # if the root node gives a feasible solution
        if testPrime(m.x):
            return (1, m.objval, m.x, 0)
#         else:
#             print("Relaxation solution at root node is ", m.x)
    
    #Intializing global upper bound and incumbent
    global_ub = 1e12
    incumbent = []
    nodes_model = [m]
    
    #Node count
    node_count = 1
    
    #------------------------------------
    # Starting branch-and-bound iteration
    #------------------------------------ 

    while( len(nodes_model) != 0 and time.time()- start_time <= time_limit):

        #------------------------------------------------------
        #Depth first search: Last node added to the list is choosen
        #-------------------------------------------------------
        m = nodes_model[-1]
        m.update()
        m.optimize()

        # prune the node by infeasiblility or pruned by bound
        if m.status != 2:
            #Removing parent node
            nodes_model.pop(-1)
            continue
        elif m.objval >= global_ub:
            #Removing parent node
            nodes_model.pop(-1)
            continue
        else:
            # update the global upper bound
            if m.objval < global_ub and testPrime(m.x):
                global_ub = m.objval
                incumbent = m.x
                # prune by optimality
                nodes_model.pop(-1)
                continue
            else:
                # find branching variable
                br_ind = int(PrimeBranching(m.x, name = branch_name, arg = [n,remainder]))
                branch_down, branch_up = Branch_val(m.x, br_ind, name = branch_name, arg = [n,remainder])
            
                # subproblem 1 branch down
                left_model = m.copy()
                left_model.getVars()[br_ind].ub = branch_down
                left_model.update()
        
                # subproblem 2 branch up
                right_model = m.copy()
                right_model.getVars()[br_ind].lb = branch_up
                right_model.update()
                
                #Removing parent node
                nodes_model.pop(-1)
        
                # Stored the node      
                nodes_model.append(right_model)
                nodes_model.append(left_model)
        
                node_count += 2
            
    end_time = time.time()
    computation_time = end_time - start_time
        
    return (node_count, global_ub, incumbent, computation_time)

In [9]:
def main_algorithm(n = 8, M = 1000, TIME_LIMIT = 3000, solve_time_limit = 600, alpha = 0.2, fixing_name = 'Naive', branching_name = 'Naive', br_arg = 0, fix_arg = [], df = None):
    start_main_time = time.time() 
    # update_time = 0
    initial_bound = M
    x = []
    m = 1
    while m <= n:
        node_count, global_ub, incumbent, computation_time = find_average_prime(n=m, init_bound = initial_bound, upper_bound=M, time_limit = solve_time_limit, alpha = alpha, branch_name=branching_name, remainder=br_arg, fix_name=fixing_name, prev_sol=x, exclude_pos=fix_arg)
        # if feasible solution found
        if incumbent != []:
            x = incumbent[:m]
            y = [int(ele) for ele in x]
            if m == n:
                # Format time
                t = round(time.time() - start_main_time, 0)
                # Format branching and arg
                if fixing_name == 'Naive' or fixing_name == 'SelectAll':
                    fix_name_tab = fixing_name
                    if branching_name == 'Naive':
                        br_name_tab = 'Naive'
                        br_arg_tab = '$\phi$'
                    elif branching_name == 'Modulo' and br_arg == 1:
                        br_name_tab = 'Modulo'
                        br_arg_tab = br_arg
                    else:
                        br_name_tab = ''
                        br_arg_tab = br_arg
                    # Format fixing
                    fix_arg_tab = '$\phi$'
                elif fixing_name == 'ExcludeOne':
                    fix_arg_tab = [i+1 for i in fix_arg]
                    if fix_arg == [0]:
                        br_name_tab = branching_name
                        if branching_name == 'Naive':
                            br_arg_tab = '$\phi$'
                        else:
                            br_arg_tab = br_arg
                        fix_name_tab = fixing_name
                    else:
                        br_name_tab = ''
                        br_arg_tab = ''
                        fix_name_tab = ''
                elif fixing_name == 'ExcludeTwo':
                    fix_arg_tab = [i+1 for i in fix_arg]
                    if fix_arg == [0,1]:
                        br_name_tab = branching_name
                        if branching_name == 'Naive':
                            br_arg_tab = '$\phi$'
                        else:
                            br_arg_tab = br_arg
                        fix_name_tab = fixing_name
                    else:
                        br_name_tab = ''
                        br_arg_tab = ''
                        fix_name_tab = ''
                df = df.append(pd.Series([br_name_tab, br_arg_tab, fix_name_tab, fix_arg_tab, m, y, t], index=df.columns), ignore_index=True)
                return df
            m += 1
        else:
            M = M * (1 + alpha)
        # Check if solve_time_limit is exceeded
        if time.time() - start_main_time > TIME_LIMIT:
            # Format branching and arg
            if fixing_name == 'Naive' or fixing_name == 'SelectAll':
                fix_name_tab = fixing_name
                if branching_name == 'Naive':
                    br_name_tab = 'Naive'
                    br_arg_tab = '$\phi$'
                elif branching_name == 'Modulo' and br_arg == 1:
                    br_name_tab = 'Modulo'
                    br_arg_tab = br_arg
                else:
                    br_name_tab = ''
                    br_arg_tab = br_arg
                # Format fixing
                fix_arg_tab = '$\phi$'
            elif fixing_name == 'ExcludeOne':
                fix_arg_tab = [i+1 for i in fix_arg]
                if fix_arg == [0]:
                    br_name_tab = branching_name
                    if branching_name == 'Naive':
                        br_arg_tab = '$\phi$'
                    else:
                        br_arg_tab = br_arg
                    fix_name_tab = fixing_name
                else:
                    br_name_tab = ''
                    br_arg_tab = ''
                    fix_name_tab = ''
            elif fixing_name == 'ExcludeTwo':
                fix_arg_tab = [[i+1,j+1] for [i, j] in fix_arg]
                if fix_arg == [0,1]:
                    br_name_tab = branching_name
                    if branching_name == 'Naive':
                        br_arg_tab = '$\phi$'
                    else:
                        br_arg_tab = br_arg
                    fix_name_tab = fixing_name
                else:
                    br_name_tab = ''
                    br_arg_tab = ''
                    fix_name_tab = ''
            df = df.append(pd.Series([br_name_tab, br_arg_tab, fix_name_tab, fix_arg_tab, m, y, t], index=df.columns), ignore_index=True)
            return df
    return df

In [10]:
if __name__ == '__main__':
    df = pd.DataFrame(columns=['Branching_strategy','Branching_arg', 'Fixing_strategy', 'Fixing_arg', '$n$', 'Solution', 'Time'])
    # Parameters
    n = 5
    M = 1000
    TIME_LIMIT = 600
    solve_time_limit = 300
    fixing_str = ['Naive', 'SelectAll', 'ExcludeOne', 'ExcludeTwo']
    branch_str = ['Naive', 'Modulo', 'Modulo', 'Modulo', 'Modulo']
    branch_arg = [0, 1, 5, 7, 11]
    # Run naive fixing
    for i in range(5):
        df = main_algorithm(n = n, M = M, TIME_LIMIT = TIME_LIMIT, solve_time_limit = solve_time_limit, alpha = 0.2, fixing_name = fixing_str[0], branching_name = branch_str[i], br_arg = branch_arg[i], fix_arg = [], df = df)
    # # Run SelectAll
    for i in range(5):
        df = main_algorithm(n = n, M = M, TIME_LIMIT = TIME_LIMIT, solve_time_limit = solve_time_limit, alpha = 0.2, fixing_name = fixing_str[1], branching_name = branch_str[i], br_arg = branch_arg[i], fix_arg = [], df = df)
    # # Run ExcludeOne
    fixing_arg = [[i] for i in range(n)]
    for i in range(5):
        for j in range(len(fixing_arg)):
            df = main_algorithm(n = n, M = M, TIME_LIMIT = TIME_LIMIT, solve_time_limit = solve_time_limit, alpha = 0.2, fixing_name = fixing_str[2], branching_name = branch_str[i], br_arg = branch_arg[i], fix_arg = fixing_arg[j], df = df)
    # Run ExcludeTwo
    fixing_arg = [[i,j] for i in range(n) for j in range(i+1, n)]
    for i in range(5):
        for j in range(len(fixing_arg)):
            df = main_algorithm(n = n, M = M, TIME_LIMIT = TIME_LIMIT, solve_time_limit = solve_time_limit, alpha = 0.2, fixing_name = fixing_str[3], branching_name = branch_str[i], br_arg = branch_arg[i], fix_arg = fixing_arg[j], df = df)
    
    # determining the name of the file
    file_name = 'average_prime_format_'+str(n)+'.xlsx'
    # saving the excel
    df.to_excel(file_name)